# Weather and Avalanches

Can we tell whether snow depth and/or snowfall are predictors of avalanche activity?

---

In [131]:
import pandas as pd

avy = pd.read_pickle("./data-prep/pkl/avy_obs.pkl")
wx = pd.read_pickle("./data-prep/pkl/all_wx.pkl")

### Consider Path 1163

In [132]:
path1163 = avy.loc[avy.avpathid == 'PATH1163']
path1163.sample(4)

,avpathid,avobsdate,avobstime,avobserverid,avtype,avtriggertyp,avtrigger,avrelativesize,avdestructivesize,avinterface,...,deb_hwy,deb_hwy_w,deb_hwy_d,avcomment,edcr_user,edcr_time,edcr_ip,eded_user,eded_time,eded_ip
avrid,,,,,,,,,,,,,,,,,,,,,
181,PATH1163,2014-01-10,14:12:00,stein_01,SS,5.0,NU,R1,D1.5,I,...,False,0.0,0.0,Terminated 2/3rds path in main path of Path 1163,Ted,2016-03-17 11:50:56,10.173.132.4,NaN,0000-00-00 00:00:00,NaN
144,PATH1163,2006-02-25,01:10:00,stein_01,SS,1.0,NaN,R3,NaN,NaN,...,False,0.0,0.0,NaN,NaN,2016-03-17 11:50:56,NaN,NaN,0000-00-00 00:00:00,NaN
157,PATH1163,2006-01-14,14:30:00,stein_01,NaN,NaN,NaN,NaN,NaN,NaN,...,False,0.0,0.0,NaN,NaN,2016-03-17 11:50:56,NaN,NaN,0000-00-00 00:00:00,NaN
50,PATH1163,2010-12-11,16:00:00,stein_01,SS,5.0,N,R2,D2,U,...,False,0.0,0.0,Debris terminated at nick point- 50% of path.,ed,2016-03-17 11:50:56,10.173.132.2,NaN,0000-00-00 00:00:00,NaN


The avobstime column needs to be combined with avobsdate to form a datetime

In [137]:
path1163['time'] = pd.to_datetime(path1163['avobstime'], format='%H:%M:%S').dt.time
path1163['dt'] = [date.replace(hour=time.hour, minute=time.minute) for date, time in zip(path1163.avobsdate, path1163.time)]
path1163.sample(4)

C:\Users\Craig\AppData\Local\Temp\ipykernel_16528\3285778417.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  path1163['time'] = pd.to_datetime(path1163['avobstime'], format='%H:%M:%S').dt.time
C:\Users\Craig\AppData\Local\Temp\ipykernel_16528\3285778417.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  path1163['dt'] = [date.replace(hour=time.hour, minute=time.minute) for date, time in zip(path1163.avobsdate, path1163.time)]


,avpathid,avobsdate,avobstime,avobserverid,avtype,avtriggertyp,avtrigger,avrelativesize,avdestructivesize,avinterface,...,edcr_time,edcr_ip,eded_user,eded_time,eded_ip,time,dt,wx_dt,snow_depth,snowfall_24hr
avrid,,,,,,,,,,,,,,,,,,,,,
30,PATH1163,2011-12-29,13:00:00,dunda_01,SS,5.0,N,R1,D2,U,...,2016-03-17 11:50:56,ed,NaN,0000-00-00 00:00:00,NaN,13:00:00,2011-12-29 13:00:00,NaT,NaN,NaN
385,PATH1163,2020-02-28,16:00:00,clark_01,WL,5.0,NU,R1,D1.5,O,...,2020-03-03 16:58:02,139.51.15.19,NaN,0000-00-00 00:00:00,NaN,16:00:00,2020-02-28 16:00:00,2020-02-28 16:00:00,70.8,0.0
402,PATH1163,2014-03-06,16:02:00,stein_01,SS,1.0,AX,R1,D1.5,S,...,2020-05-17 04:01:50,64.91.61.64,Ted,2020-05-22 12:39:30,64.91.61.64,16:02:00,2014-03-06 16:02:00,NaT,NaN,NaN
381,PATH1163,2020-02-01,08:05:00,clark_01,SS,5.0,NU,R1,D2,O,...,2020-02-01 14:29:21,139.51.15.19,NaN,0000-00-00 00:00:00,NaN,08:05:00,2020-02-01 08:05:00,2020-02-01 08:00:00,61.1,0.4


Use the Shed 11 Wx station to get snow depth and snowfall in past 24 hours

In [138]:
shed11 = wx.loc[wx.site == 'S11MT']
shed11.head(4)

,dt,temp,dew_pt,rH,heat_idx,wind_chill,wind_dir,wind_spd,hr_precip,snow_depth,...,snowfall_6hr,snowfall_24hr,site,file_name,year,sea_lvl_press,sta_press,altimeter_setting,solar_radiation,pct_possible
26167,2022-12-23 04:00:00,-10.0,-15.0,78.0,NaN,NaN,W,2G4,NaN,54.0,...,0.0,0.0,S11MT,S11MT-2022.html,2022,NaN,NaN,NaN,NaN,NaN
26168,2022-12-23 03:00:00,-11.0,-16.0,78.0,NaN,NaN,SW,2G5,NaN,54.1,...,0.0,0.0,S11MT,S11MT-2022.html,2022,NaN,NaN,NaN,NaN,NaN
26169,2022-12-23 02:00:00,-12.0,-17.0,78.0,NaN,NaN,WSW,2G5,NaN,54.4,...,0.0,0.0,S11MT,S11MT-2022.html,2022,NaN,NaN,NaN,NaN,NaN
26170,2022-12-23 01:00:00,-13.0,-18.0,78.0,NaN,NaN,SW,3G7,NaN,54.5,...,0.0,0.0,S11MT,S11MT-2022.html,2022,NaN,NaN,NaN,NaN,NaN


The tricky part. For each avalanche observation, we find the closest wx observation (that happened before the avalanche).

(Sometimes this doesn't exist because the avalanche was recorded before Shed 11 was capturing wx data.)

We appead the datetime of the wx observation, the snow depth and snowfall in previous 24 hrs. to the avy obs record.

In [139]:
def getWx(avy_dt):
    prev_wx_obs = shed11.loc[shed11.dt <= avy_dt].sort_values(by=['dt'], ascending=False).head(1)
    if len(prev_wx_obs) > 0:
        return prev_wx_obs.iloc[0, 0]

def getSnowDepth(avy_dt):
    prev_wx_obs = shed11.loc[shed11.dt <= avy_dt].sort_values(by=['dt'], ascending=False).head(1)
    if len(prev_wx_obs) > 0:
        return prev_wx_obs.iloc[0, 9]

def getSnowfall24Hr(avy_dt):
    prev_wx_obs = shed11.loc[shed11.dt <= avy_dt].sort_values(by=['dt'], ascending=False).head(1)
    if len(prev_wx_obs) > 0:
        return prev_wx_obs.iloc[0, 12]

path1163['wx_dt'] = [getWx(avy_dt) for avy_dt in path1163.dt]
path1163['snow_depth'] = [getSnowDepth(avy_dt) for avy_dt in path1163.dt]
path1163['snowfall_24hr'] = [getSnowfall24Hr(avy_dt) for avy_dt in path1163.dt]

path1163

C:\Users\Craig\AppData\Local\Temp\ipykernel_16528\1591971095.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  path1163['wx_dt'] = [getWx(avy_dt) for avy_dt in path1163.dt]
C:\Users\Craig\AppData\Local\Temp\ipykernel_16528\1591971095.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  path1163['snow_depth'] = [getSnowDepth(avy_dt) for avy_dt in path1163.dt]
C:\Users\Craig\AppData\Local\Temp\ipykernel_16528\1591971095.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

,avpathid,avobsdate,avobstime,avobserverid,avtype,avtriggertyp,avtrigger,avrelativesize,avdestructivesize,avinterface,...,edcr_time,edcr_ip,eded_user,eded_time,eded_ip,time,dt,wx_dt,snow_depth,snowfall_24hr
avrid,,,,,,,,,,,,,,,,,,,,,
2,PATH1163,2012-03-16,08:30:00,stein_01,U,5.0,NU,U,D2,U,...,2016-03-17 11:50:56,10.173.132.2,NaN,0000-00-00 00:00:00,NaN,08:30:00,2012-03-16 08:30:00,NaT,NaN,NaN
12,PATH1163,2012-02-22,12:40:00,stein_01,SS,5.0,N,R2,D2,O,...,2016-03-17 11:50:56,10.173.132.2,NaN,0000-00-00 00:00:00,NaN,12:40:00,2012-02-22 12:40:00,NaT,NaN,NaN
14,PATH1163,2012-02-22,08:15:00,stein_01,U,5.0,N,U,D2,U,...,2016-03-17 11:50:56,10.173.132.2,NaN,0000-00-00 00:00:00,NaN,08:15:00,2012-02-22 08:15:00,NaT,NaN,NaN
30,PATH1163,2011-12-29,13:00:00,dunda_01,SS,5.0,N,R1,D2,U,...,2016-03-17 11:50:56,ed,NaN,0000-00-00 00:00:00,NaN,13:00:00,2011-12-29 13:00:00,NaT,NaN,NaN
42,PATH1163,2011-01-16,09:00:00,stein_01,U,5.0,NU,R2,D2,U,...,2016-03-17 11:50:56,01,NaN,0000-00-00 00:00:00,NaN,09:00:00,2011-01-16 09:00:00,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,PATH1163,2022-02-10,14:49:00,clark_01,WL,5.0,NU,R1,D2,O,...,2022-02-11 16:51:42,170.49.113.19,Adam,2022-02-17 18:45:51,170.49.113.19,14:49:00,2022-02-10 14:49:00,2022-02-10 14:00:00,64.6,0.0
491,PATH1163,2022-02-20,09:45:00,clark_01,L,5.0,NU,R1,D1.5,S,...,2022-02-20 13:16:16,170.49.113.17,NaN,0000-00-00 00:00:00,NaN,09:45:00,2022-02-20 09:45:00,2022-02-20 09:00:00,72.6,7.6
494,PATH1163,2022-03-01,07:35:00,clark_01,U,5.0,NU,U,D2.5,I,...,2022-03-01 13:32:54,170.49.113.17,Adam,2022-03-15 13:34:54,10.30.72.4,07:35:00,2022-03-01 07:35:00,2022-03-01 07:00:00,79.6,8.4


We now have a dataframe with all the original Path 1163 avy obs data, plus the nearest wx data (if it was available).

We could append more columns of weather data if we want...